In [53]:
import pandas as pd
import numpy as np
import re
import math

In [54]:
df = pd.read_csv("bajuPria_enriched.csv")
print(df.shape)
df.head()

(1859, 20)


,id,name,url,category_breadcrumb,price_number,price_original,discountPercentage,mediaURL_image,ratingAverage,shop_id,shop_name,shop_url,shop_city,shop_tier,countSold,isTopAds,labelGroups,label_titles,totalRating,countReview
0,102562986014,Indostreet - Kaos Lengan Pendek Logo One Sampi...,https://www.tokopedia.com/indostreet/indostree...,fashion-pria/atasan-pria/kaos-pria,66000,Rp200.000,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.8,7495885842339695079,INDOSTREET_NEW,https://www.tokopedia.com/indostreet,Jakarta Pusat,2,250.0,False,"[{""position"": ""ri_product_credibility"", ""title...",250+ terjual | Rp66.000,76.0,57.0
1,100112974607,Paket 4PCS 100RIBU Kaos Distro BAHAN SEMI KATU...,https://www.tokopedia.com/dewa-muraah/paket-4p...,fashion-pria/atasan-pria/kaos-pria,89900,Rp100.000,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.8,7494664242693770044,Dewa Muraah,https://www.tokopedia.com/dewa-muraah,Tangerang,3,100000.0,False,"[{""position"": ""ri_product_credibility"", ""title...",100rb+ terjual | Rp89.900,41756.0,19657.0
2,102116827290,T-shirt RUCAS ldistro pria/wanitapremium black...,https://www.tokopedia.com/fashion-good-t-shiet...,fashion-pria/atasan-pria/kaos-pria,90000,Rp400.000,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.4,7496322972430993787,fashion good t-shiet,https://www.tokopedia.com/fashion-good-t-shiet,Kab. Tangerang,1,10000.0,False,"[{""position"": ""ri_product_credibility"", ""title...",10rb+ terjual | Rp90.000,1090.0,424.0
3,12907925502,No Void Minds FORMLOCK Oversized T-Shirt | Oat...,https://www.tokopedia.com/novoidminds/no-void-...,fashion-pria/atasan-pria/kaos-pria,99900,NaN,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.9,5976141,No Void Minds Store,https://www.tokopedia.com/novoidminds,Bandung,2,10000.0,False,"[{""position"": ""ri_product_credibility"", ""title...",10rb+ terjual | Rp99.900,7001.0,2002.0
4,14497861959,Kaos polo shirt polos pria termurah | atasan b...,https://www.tokopedia.com/mensclothing-1/kaos-...,fashion-pria/atasan-pria/polo-shirt-pria,219000,NaN,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.7,17747484,Mens Clothing1,https://www.tokopedia.com/mensclothing-1,Kab. Bandung,1,2000.0,False,"[{""position"": ""ri_product_credibility"", ""title...",2rb+ terjual | Rp219.000,881.0,256.0


In [55]:
# Cek duplikat berdasarkan kolom 'id'
duplicatedDataById = df['id'].duplicated().sum()
print("Duplicated rows based on 'id':", duplicatedDataById)

Duplicated rows based on 'id': 0


In [ ]:
# Drop duplikat kalau ada
# df = df.drop_duplicates(subset=["id"])

In [56]:
# untuk membersihkan teks
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()                             # Jadikan semua huruf kecil
    text = re.sub(r"[^a-z0-9\s]", " ", text)        # Ganti semua karakter selain huruf a-z, angka 0-9, dan spasi menjadi spasi
    text = re.sub(r"\s+", " ", text).strip()        # Rapikan spasi berlebihan jadi satu spasi, lalu hapus spasi di awal/akhir
    return text

In [57]:
df["name_clean"] = df["name"].apply(clean_text)
df["city_clean"] = df["shop_city"].apply(clean_text)

In [58]:
# normalisasi kolom price_original
df["price_original"] = (
    df["price_original"]
        .astype("string")
        .str.replace(r"[^0-9]", "", regex=True)
        .replace("", pd.NA)
        .astype("float")
        .astype("Int64")
)

In [59]:
normal_col = "price_number"  

# Hitung discountPercentage
df["discountPercentage"] = np.where(
    df["price_original"].notna() & (df["price_original"] > 0),
    ((df["price_original"] - df[normal_col]) / df["price_original"] * 100),
    0   # ← kalau price_original NaN / 0
)

# Rapikan hasil
df["discountPercentage"] = (
    df["discountPercentage"]
        .round(0)        # bisa ganti round(2)
        .clip(0, 100)    # jaga tetap masuk akal
        .astype(int)     # opsional: jadi integer
)

# cek hasil
df[[normal_col, "price_original", "discountPercentage"]].head(10)


,price_number,price_original,discountPercentage
0,66000,200000,67
1,89900,100000,10
2,90000,400000,78
3,99900,<NA>,0
4,219000,<NA>,0
5,72900,<NA>,0
6,69900,139800,50
7,100000,200000,50
8,89900,<NA>,0
9,50000,130000,62


In [61]:
# ubah ke int
cols = ["countSold", "countReview", "totalRating"]

df[cols] = df[cols].round(0).astype("Int64")

In [62]:
df[["countSold", "countReview", "totalRating"]].head(10)

,countSold,countReview,totalRating
0,250,57,76
1,100000,19657,41756
2,10000,424,1090
3,10000,2002,7001
4,2000,256,881
5,1000,98,369
6,50000,6209,9898
7,100000,8350,13718
8,100000,3376,15196
9,50000,216,554


In [63]:
df.head()

,id,name,url,category_breadcrumb,price_number,price_original,discountPercentage,mediaURL_image,ratingAverage,shop_id,...,shop_city,shop_tier,countSold,isTopAds,labelGroups,label_titles,totalRating,countReview,name_clean,city_clean
0,102562986014,Indostreet - Kaos Lengan Pendek Logo One Sampi...,https://www.tokopedia.com/indostreet/indostree...,fashion-pria/atasan-pria/kaos-pria,66000,200000,67,https://p16-images-sign-sg.tokopedia-static.ne...,4.8,7495885842339695079,...,Jakarta Pusat,2,250,False,"[{""position"": ""ri_product_credibility"", ""title...",250+ terjual | Rp66.000,76,57,indostreet kaos lengan pendek logo one samping...,jakarta pusat
1,100112974607,Paket 4PCS 100RIBU Kaos Distro BAHAN SEMI KATU...,https://www.tokopedia.com/dewa-muraah/paket-4p...,fashion-pria/atasan-pria/kaos-pria,89900,100000,10,https://p16-images-sign-sg.tokopedia-static.ne...,4.8,7494664242693770044,...,Tangerang,3,100000,False,"[{""position"": ""ri_product_credibility"", ""title...",100rb+ terjual | Rp89.900,41756,19657,paket 4pcs 100ribu kaos distro bahan semi katu...,tangerang
2,102116827290,T-shirt RUCAS ldistro pria/wanitapremium black...,https://www.tokopedia.com/fashion-good-t-shiet...,fashion-pria/atasan-pria/kaos-pria,90000,400000,78,https://p16-images-sign-sg.tokopedia-static.ne...,4.4,7496322972430993787,...,Kab. Tangerang,1,10000,False,"[{""position"": ""ri_product_credibility"", ""title...",10rb+ terjual | Rp90.000,1090,424,t shirt rucas ldistro pria wanitapremium black...,kab tangerang
3,12907925502,No Void Minds FORMLOCK Oversized T-Shirt | Oat...,https://www.tokopedia.com/novoidminds/no-void-...,fashion-pria/atasan-pria/kaos-pria,99900,<NA>,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.9,5976141,...,Bandung,2,10000,False,"[{""position"": ""ri_product_credibility"", ""title...",10rb+ terjual | Rp99.900,7001,2002,no void minds formlock oversized t shirt oatmeal,bandung
4,14497861959,Kaos polo shirt polos pria termurah | atasan b...,https://www.tokopedia.com/mensclothing-1/kaos-...,fashion-pria/atasan-pria/polo-shirt-pria,219000,<NA>,0,https://p16-images-sign-sg.tokopedia-static.ne...,4.7,17747484,...,Kab. Bandung,1,2000,False,"[{""position"": ""ri_product_credibility"", ""title...",2rb+ terjual | Rp219.000,881,256,kaos polo shirt polos pria termurah atasan baj...,kab bandung


In [64]:
df.columns

Index(['id', 'name', 'url', 'category_breadcrumb', 'price_number',
       'price_original', 'discountPercentage', 'mediaURL_image',
       'ratingAverage', 'shop_id', 'shop_name', 'shop_url', 'shop_city',
       'shop_tier', 'countSold', 'isTopAds', 'labelGroups', 'label_titles',
       'totalRating', 'countReview', 'name_clean', 'city_clean'],
      dtype='object')

In [65]:
UMKM_KEYWORDS = [
    "umkm", "lokal", "handmade", "kerajinan",
    "konveksi", "rumahan", "home industry",
    "custom", "distro"
]

In [66]:
# isTopAds: NA → False
df["isTopAds"] = df["isTopAds"].fillna(False)

# shop_tier: NA → angka besar (anggap non-UMKM)
df["shop_tier"] = df["shop_tier"].fillna(99)

# countSold: NA → 0
df["countSold"] = df["countSold"].fillna(0)

# label_titles: NA → string kosong
df["label_titles"] = df["label_titles"].fillna("")

# name_clean & shop_name
df["name_clean"] = df["name_clean"].fillna("")
df["shop_name"] = df["shop_name"].fillna("")


In [68]:
def umkm_score(row):
    score = 0

    # 1️⃣ Penjualan kecil
    if int(row["countSold"]) < 50000:
        score += 1

    # 2️⃣ Tidak TopAds
    if row["isTopAds"] is False:
        score += 1

    # 3️⃣ Tier toko rendah
    if int(row["shop_tier"]) <= 2:
        score += 1

    # 4️⃣ Label lokal
    if "beli lokal" in row["label_titles"]:
        score += 1

    # 5️⃣ Keyword UMKM
    if any(
        k in row["name_clean"] or k in row["shop_name"]
        for k in UMKM_KEYWORDS
    ):
        score += 1

    return score

In [69]:
df["umkm_score"] = df.apply(umkm_score, axis=1)
df["is_umkm"] = df["umkm_score"] >= 3

In [70]:
df["umkm_score"].describe()
df["is_umkm"].value_counts()

is_umkm
True     1403
False     456
Name: count, dtype: int64

In [ ]:
def umkm_score(row):
    score = 0

    # 1️⃣ Penjualan kecil
    if row.get("countSold", 0) < 50000:
        score += 1

    # 2️⃣ Tidak TopAds
    if row.get("isTopAds") is False:
        score += 1

    # 3️⃣ Tier toko rendah
    if row.get("shop_tier", 99) <= 2:
        score += 1

    # 4️⃣ Label lokal
    if "beli lokal" in row.get("label_titles", ""):
        score += 1

    # 5️⃣ Keyword UMKM di nama/toko
    name = row.get("name_clean", "")
    shop = row.get("shop_name", "")
    if any(k in name or k in shop for k in UMKM_KEYWORDS):
        score += 1

    return score

In [ ]:
df.head()

In [ ]:
df["umkm_score"] = df.apply(umkm_score, axis=1)
df["is_umkm"] = df["umkm_score"] >= 3

In [ ]:
# log transform harga
df["price_log"] = np.log(df["price_number"] + 1)

In [ ]:
df["price_log"].head(10)

In [ ]:
# normalisasi rating & diskon
df["rating_norm"] = (df["ratingAverage"].fillna(0).clip(0, 5)) / 5
df["discount_norm"] = (df["discountPercentage"].fillna(0).clip(0, 100)) / 100

In [ ]:
df[["rating_norm", "discount_norm"]].head(10)

In [ ]:
# normalisasi popularity
sold = df["countSold"].fillna(0)
rev  = df["countReview"].fillna(0)

df["sold_log"] = np.log(sold + 1)
df["review_log"] = np.log(rev + 1)

df["popularity"] = 0.5 * df["sold_log"] + 0.5 * df["review_log"]


In [ ]:
df[["sold_log", "review_log", "popularity"]].head(10)

In [ ]:
df[["shop_tier", "labelGroups"]].head()

In [ ]:
import json, ast

def extract_label_titles(labelGroups):
    """
    Mengembalikan list title dari labelGroups.
    labelGroups bisa berupa: list[dict], string JSON, atau NaN.
    """
    if pd.isna(labelGroups):
        return []

    obj = labelGroups

    # kalau bentuknya string, coba parse
    if isinstance(obj, str):
        obj = obj.strip()
        if not obj:
            return []
        try:
            obj = json.loads(obj)
        except Exception:
            try:
                obj = ast.literal_eval(obj)
            except Exception:
                return []

    # sekarang obj diharapkan list/dict
    titles = []
    if isinstance(obj, list):
        for it in obj:
            if isinstance(it, dict):
                t = it.get("title")
                if t:
                    titles.append(str(t).lower())
    elif isinstance(obj, dict):
        t = obj.get("title")
        if t:
            titles.append(str(t).lower())

    return titles


In [ ]:
def umkm_score(row):
    score = 0

    # 1️⃣ Penjualan kecil
    if row["countSold"] < 50000:
        score += 1

    # 2️⃣ Tidak TopAds
    if row.get("isTopAds") == False:
        score += 1

    # 3️⃣ Tier toko rendah
    if row["shop_tier"] <= 2:
        score += 1

    # 4️⃣ Ada label lokal
    if "beli lokal" in row["label_titles"]:
        score += 1

    # 5️⃣ Keyword UMKM di nama/toko
    if any(k in row["name_clean"] or k in row["shop_name"] for k in UMKM_KEYWORDS):
        score += 1

    return score


In [ ]:
df["umkm_score"] = df.apply(umkm_score, axis=1)

In [ ]:
UMKM_KEYWORDS = [
    "umkm", "lokal", "handmade", "daerah",
    "kerajinan", "konveksi", "rumahan"
]

def label_umkm(row):
    score = 0

    # 1. Official store (jika kolom ada)
    if "is_official" in row.index:
        if row["is_official"] == False:
            score += 1
    else:
        # jika tidak ada, anggap non-official
        score += 1

    # 2. TopAds
    if "isTopads" in row.index and row["isTopads"] == False:
        score += 1

    # 3. Penjualan
    if "countSold_num" in row.index and row["countSold_num"] < 50000:
        score += 1

    # 4. Keyword UMKM
    name = str(row.get("name_clean", "")).lower()
    shop = str(row.get("shop_name", "")).lower()
    if any(k in name or k in shop for k in UMKM_KEYWORDS):
        score += 1

    return score >= 3


In [ ]:
df["umkm"] = df.apply(label_umkm, axis=1)


In [ ]:
df["umkm"].value_counts()

In [ ]:
def is_umkm(row):
    name = row.get("name", "")
    sold = row.get("countSold", 0)
    is_ads = row.get("isTopAds", False)     # kolom yang ada
    shop_tier = row.get("shop_tier", pd.NA) # angka
    labelGroups = row.get("labelGroups", pd.NA)

    # normalisasi NA
    name = "" if pd.isna(name) else str(name).lower()
    sold = 0 if pd.isna(sold) else float(sold)
    is_ads = False if pd.isna(is_ads) else bool(is_ads)

    # ekstrak label titles
    titles = extract_label_titles(labelGroups)

    # heuristik official dari labelGroups (paling “nyata” karena ada teksnya)
    official_markers = ["official", "official store", "tokopedia official", "mall"]
    is_official = any(any(m in t for m in official_markers) for t in titles)

    # fallback tambahan dari shop_tier (karena kamu punya 1 dan 3)
    # asumsi umum: tier lebih tinggi cenderung merchant besar/official
    # jadi kalau tier == 3, anggap bukan UMKM
    if (not pd.isna(shop_tier)) and int(shop_tier) >= 3:
        is_official = True

    # aturan UMKM (sesuaikan)
    keywords = ["umkm", "usaha mikro", "homemade", "rumahan", "handmade", "lokal"]
    keyword_match = any(k in name for k in keywords)

    sold_limit = sold <= 1000
    not_ads = not is_ads
    not_official = not is_official

    return bool(keyword_match and sold_limit and not_ads and not_official)


df["umkm"] = df.apply(is_umkm, axis=1)

In [ ]:
df.head()

In [ ]:
df["umkm"].value_counts()

In [ ]:
df_umkm = df[df["umkm"] == True]
df_umkm.head()